In [2]:
import gc
import numpy as np
import pandas as pd

gc.collect()

0

In [10]:
return_df = pd.read_csv('./general_results/result.csv', index_col=0).fillna(0)

In [12]:
return_df

,0,S&P 500
2010-07-02 00:00:00,0.000000,0.000000
2010-07-06 00:00:00,0.000000,0.000000
2010-07-07 00:00:00,0.000000,0.000000
2010-07-08 00:00:00,0.000000,0.000000
2010-07-09 00:00:00,0.000000,0.000000
...,...,...
2025-03-17 00:00:00,0.042407,0.006396
2025-03-18 00:00:00,0.003949,-0.010711
2025-03-19 00:00:00,-0.026934,0.010741
2025-03-20 00:00:00,0.002069,-0.002187


In [15]:
for col in return_df.columns:
    print(col)
    cl = 5  # confidence level 5%
    VaR = np.percentile(return_df[col], cl)
    ES = return_df[col][return_df[col] <= VaR].mean()

    print(f"Monte Carlo VaR (5%): {VaR:.4f}")
    print(f"Monte Carlo ES (5%): {ES:.4f}\n")

0
Monte Carlo VaR (5%): -0.0229
Monte Carlo ES (5%): -0.0374

S&P 500
Monte Carlo VaR (5%): -0.0165
Monte Carlo ES (5%): -0.0264



In [44]:
# returns: (time, assets) 형태의 과거 수익률 데이터
returns = dual_ret

# 1. 과거 수익률의 평균/공분산 계산
mean_returns = returns.mean()
cov_returns = returns.cov()

# 2. MC 샘플링
n_simulations = 10000
n_assets = returns.shape[1]

# 다변량 정규분포에서 샘플링
simulated_returns = np.random.multivariate_normal(mean_returns, cov_returns, n_simulations)

# 3. 포트폴리오 수익률 계산
# (여기서는 equal weight, 즉 각 자산 10%씩 투자한다고 가정)
weights = np.ones(n_assets) / n_assets
portfolio_returns = simulated_returns @ weights

# 4. VaR, ES 계산
cl = 5  # confidence level 5%
VaR = np.percentile(portfolio_returns, cl)
ES = portfolio_returns[portfolio_returns <= VaR].mean()

print(f"Monte Carlo VaR (5%): {VaR:.4f}")
print(f"Monte Carlo ES (5%): {ES:.4f}")

Monte Carlo VaR (5%): -0.0276
Monte Carlo ES (5%): -0.0350


In [126]:
# 설정
n_assets = len(returns.columns)
n_days = len(returns.index)
n_simulations = 100

mu = returns.mean()
sigma = returns.std()
corr_matrix = returns.corr()
cov_matrix = np.outer(sigma, sigma) * corr_matrix
L = np.linalg.cholesky(cov_matrix)

Z = np.random.normal(size=(n_days, n_assets))  # 표준 정규분포 난수
correlated_random = (L @ Z.T).T  # 상관된 난수 만들기

# 수익률 생성
returns = np.array(mu / n_days) + (correlated_random / np.sqrt(n_days))  # 드리프트 + 변동성

# 가격 경로 생성
price_paths = (1 + returns).cumprod(axis=0)